In [ ]:
# 不行啊，虽然把权值控制在了[-1,1]这个范围，但是输出只有一个神经元，怎么能按照公式那种进行每个神经元单独反馈呢 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 读取mat文件
dataPath = 'E:/tensorflow/data_set/WCDMA/wcdma.mat'
data = scio.loadmat(dataPath)
print(type(data))

In [ ]:
print(data.keys())# 打印字典的键
# train_test = data['input']
# train_test_1 = train_test[np.newaxis,:]
# print(np.shape(data['input'][np.newaxis,:,0]))

In [ ]:
# 跟着网上学的next_batch喂给placeholder
def next_batch(data, batch_size):  
    index = [ i for i in range(0,np.size(data[0]))]  
    np.random.shuffle(index)  # 打乱顺序
    batch_data = []
    for i in range(0,batch_size):  
        batch_data.append(data[:,index[i]])
    return batch_data

In [ ]:
# 参数设定
BATCH_SIZE = 16

x_train = np.real(data['input']).astype('float32')
y = 38400

tf_x = tf.placeholder(tf.float32,[None,38400])
tf_w = tf.Variable(tf.random_uniform([38400,1],minval=-1,maxval=1,dtype=tf.float32),name='seq')
prediction = tf.matmul(tf_x,tf_w)
# loss = tf.sqrt(tf.reduce_mean(tf.abs(tf.multiply(tf.matmul(tf.transpose(tf_w),tf_w),tf.transpose(tf_x))-tf.multiply(tf.matmul(tf.transpose(tf_w),tf.transpose(tf_x)),tf_w))))
loss = tf.reduce_mean(1/tf.abs(prediction))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

D_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'seq')
clip_ops = []
for var in D_vars:
    clip_bounds = [-1,1]
    clip_ops.append(
        tf.assign(
            var, 
            tf.clip_by_value(var, clip_bounds[0], clip_bounds[1])
        )
    )
clip_disc_weights = tf.group(*clip_ops)
correct_prediction = tf.equal(np.real(data['sc']).astype('float32'),tf.sign(tf_w))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(100000):
        b_x = next_batch(x_train,BATCH_SIZE)
        sess.run(train,{tf_x:b_x})
        _ = sess.run(clip_disc_weights)
        
        if i%500 == 0:
            print(i,sess.run(loss,{tf_x:b_x}),sess.run(accuracy))